# MODFLOW 6 GWF Model: Advanced Packages

The purpose of this exercise is to convert the RIV and lake CHD packages to SFR and LAK packages, respectively, and add a mover package to transfer water between the SFR and LAK packages. We will also use a time series file to define SFR inflows and add LAK and SFR observations.

In [ ]:
import os
import sys
import numpy as np
import flopy
import matplotlib as mpl
import matplotlib.pyplot as plt

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

### Load an existing model with traditional packages

In [ ]:
data_ws = '../../data/gwt_ts2/'
mname = 'gwt_ts2'
s = flopy.mf6.MFSimulation().load(sim_ws=data_ws, exe_name='mf6')

#### Get list of files in the data workspace directory

In [ ]:
os.listdir(data_ws)

#### Change the model work space

We will change the model workspace so we don't overwrite the original model files.

In [ ]:
model_ws = '../../working/advpack/'
s.set_sim_path(model_ws)

In [ ]:
# get the groundwater model
gwf = s.get_model(mname)

#### Retrieve nlay, nrow, and ncol from the groundwater flow model

In [ ]:
nlay, nrow, ncol = gwf.dis.nlay.array, gwf.dis.nrow.array, gwf.dis.ncol.array
nlay, nrow, ncol

### Get the package names

We will need the package names to get data from the existing packages representing the river and lakes in the model domain.

In [ ]:
gwf.package_names

### Plot the locations of the the river and lake boundaries

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
mm.plot_grid(lw=0.5, color='black')
mm.plot_bc('RIV')
mm.plot_bc('LAK-1', color='cyan')
mm.plot_ibound()

### Get the river location data from the existing river package

In [ ]:
riv = gwf.get_package('RIV-1')
rivspd = riv.stress_period_data.get_data(key=0)
rivspd.dtype

### Remove the existing river package

In [ ]:
# get rid of the existing river package
gwf.remove_package('RIV')

### Create the SFR model from the river location data and data in `'sfr-packagedata.dat'`

In [ ]:
# read sfr package data
sfrpd = np.genfromtxt(data_ws + 'sfr-packagedata.dat', names=True)
sfrpd.dtype

#### Create a default dtype for the SFR packagedata

In [ ]:
# create a default dtype
sfrpackagedata = flopy.mf6.ModflowGwfsfr.packagedata.empty(gwf, boundnames=True, maxbound=rivspd.shape[0])
sfrpackagedata.dtype

In [ ]:
# fill package data
for name in sfrpackagedata.dtype.names :
    if name in rivspd.dtype.names:
        sfrpackagedata[name] = rivspd[name]
sfrpackagedata

#### Fill the empty array with data read from `'sfr-packagedata.dat'`

In [ ]:
for name in sfrpackagedata.dtype.names:
    if name in sfrpd.dtype.names:
        sfrpackagedata[name] = sfrpd[name]
sfrpackagedata['boundnames'] = rivspd['boundname']    
sfrpackagedata    

In [ ]:
type(sfrpackagedata['cellid'][0])

### Read SFR connection data from `'sfr-connectiondata.dat'`

In [ ]:
with open(data_ws + 'sfr-connectiondata.dat') as f: 
    lines = f.readlines()
sfrconnectiondata = []
for line in lines:
    t = line.split()
    c = []
    for v in t:
        i = int(v)
        c.append(i)
    sfrconnectiondata.append(c)

#### Load time series that can be used to specify inflows

In [ ]:
# load inflow timeseries data
inflow_data = np.loadtxt(data_ws + 'sfr_timeseries.dat')

#### Calculate the mean inflow

In [ ]:
mean_inflow = inflow_data[:, 1].mean()
mean_inflow

#### Plot the time series data

In [ ]:
plt.plot(inflow_data[:, 0], inflow_data[:, 1])

#### Create a list of tuples for the inflow data

In [ ]:
inflow_data = list(map(tuple, inflow_data))
#inflow_data

#### Create SFR inflows

In [ ]:
sfrperioddata = {0: [[0, 'inflow', mean_inflow]]}
time_series_dict = None

#### Class exercise 1

Uncomment cell block below to use a time varying inflow rate

In [ ]:
# sfrperioddata = {0: [[0, 'inflow', 'inflow_rate']]}
# time_series_dict = {'filename': 'sfr-inflow-rates.ts',
#                     'timeseries': inflow_data,
#                     'time_series_namerecord': 'inflow_rate',
#                     'interpolation_methodrecord': 'linear'}

#### Add the SFR model to the GWF model

In [ ]:
sfr = flopy.mf6.ModflowGwfsfr(gwf, 
                              stage_filerecord=mname+'.sfr.stage.bin', 
                              budget_filerecord=mname+'.sfr.cbc', 
                              mover=True, pname='SFR-1',
                              unit_conversion=128390.00, 
                              boundnames=True, nreaches=38,
                              packagedata=sfrpackagedata, 
                              connectiondata=sfrconnectiondata,
                              perioddata=sfrperioddata,
                              timeseries=time_series_dict)

In [ ]:
sfrobsname = mname + '.sfr.obs'
sfr_obs = [('SFR06-S', 'STAGE', 5), ('SFR06-Q', 'DOWNSTREAM-FLOW', 5),
           ('SFR07-S', 'STAGE', 6), ('SFR07-Q', 'DOWNSTREAM-FLOW', 6),
           ('SFR38-S', 'STAGE', 37), ('SFR38-Q', 'DOWNSTREAM-FLOW', 37),
           ('SEG01', 'SFR', 'SEG1'), ('SEG02', 'SFR', 'SEG2'),
           ('SEG03', 'SFR', 'SEG3'), ('SEG04', 'SFR', 'SEG4')]
sfr.obs.initialize(filename=sfrobsname, continuous={sfrobsname + '.csv': sfr_obs})

### Create the Lake package

Get the cellids for the lake from the CHD package that currently defines the lakes

In [ ]:
lakchd = gwf.get_package('LAK-1')
chdspd = lakchd.stress_period_data.get_data(key=0)
chdloc = chdspd['cellid']
chdloc

#### Remove the CHD package representing the lake

In [ ]:
gwf.remove_package('LAK-1')

#### Read the lake packagedata from `'lak-connectiondata.dat'`

In [ ]:
# read lak package data
dtype = [('lakeno', np.int32), ('iconn', np.int32), 
         ('k', np.int32), ('i', np.int32), ('j', np.int32), 
         ('claktype', '|U20'), ('bedleak', np.float), ('belev', np.float), 
         ('telev', np.float), ('connlen', np.float), ('connwidth', np.float)]
lakpd = np.genfromtxt(data_ws + 'lak-connectiondata.dat', dtype=dtype)
lakpd.dtype

#### Create cellids for the lakes

In [ ]:
arr = np.column_stack((lakpd['k'], lakpd['i'], lakpd['j']))
cellid = tuple(map(tuple, arr))
len(cellid)

#### Create Lake connectiondata

In [ ]:
lakeconnectiondata = flopy.mf6.ModflowGwflak.connectiondata.empty(gwf, maxbound=len(cellid))
for name in lakeconnectiondata.dtype.names:
    if name == 'cellid':
        lakeconnectiondata[name] = cellid
    else:
        lakeconnectiondata[name] = lakpd[name]
lakeconnectiondata

#### Get the unique lake numbers

In [ ]:
uniquelakes = np.unique(lakpd['lakeno'])
uniquelakes

In [ ]:
nlakecon = np.zeros(uniquelakes.shape, dtype=np.int)
for lak in lakpd['lakeno']:
    for idx, lid in enumerate(uniquelakes):
        if lid == lak:
            nlakecon[idx] += 1
nlakecon

In [ ]:
lakpackagedata = [[0, 44., nlakecon[0], 'lake1'],
                  [1, 35.2, nlakecon[1], 'lake2']]

#### Create outlet data

In [ ]:
outlet=[[0, 0, 0, 'MANNING', 44.5, 5.000000, 0.03,  0.2187500E-02]]

#### Create the lake package

In [ ]:
lak = flopy.mf6.ModflowGwflak(gwf, pname='LAK-1', time_conversion=86400.000, 
                              mover=True, boundnames=True, 
                              nlakes=2, noutlets=1,
                              outlets=outlet, 
                              packagedata=lakpackagedata, 
                              connectiondata=lakeconnectiondata)

#### Create lake observations

In [ ]:
lakobsname = mname + '.lak.obs'
lak_obs = [('LAK1-S', 'STAGE', 1),
           ('LAK2-S', 'STAGE', 2),
           ('LAK1-Q', 'LAK', 'LAKE1'), 
           ('LAK2-Q', 'LAK', 'LAKE2')]
lak.obs.initialize(filename=lakobsname, continuous={lakobsname+'.csv': lak_obs})

#### Class exercise 2

Empty code blocks for creating the MAW package and MAW observations

-----------------

Make the `packagedata`

Make the `connectiondata`

Make the stress period data

#### Create the MAW package

Set the package name (`pname`) to `MAW-1`. Also create observations and add to the MAW package (`obs.initialize()`)

### Reset idomain using CHD package locations representing the lakes

In [ ]:
idomain = gwf.dis.idomain.array
for loc in chdloc:
    idomain[loc] = 0
gwf.dis.idomain.set_data(idomain[0], layer=0, multiplier=[1])

### Create the MVR package

Connect SFR reach 6 to lake 1 and connect lake 1 to SFR reach 7

In [ ]:
mvrperioddata = [['SFR-1', 5, 'LAK-1', 0, 'FACTOR',  1.],
                 ['LAK-1', 0, 'SFR-1', 6, 'FACTOR',  1.]]
maxmvr, maxpackages = 2, 2
mvrpack = [['SFR-1'], ['LAK-1']]

#### Class exercise 3

Empty code blocks for SFR, LAK, and MAW package connections to other packages using the MVR package

In [ ]:
mvr = flopy.mf6.ModflowGwfmvr(gwf, maxmvr=maxmvr, maxpackages=maxpackages, 
                              packages=mvrpack, 
                              perioddata=mvrperioddata)

### Write the MODFLOW 6 files

In [ ]:
s.write_simulation()
s.run_simulation()

### Load sfr and lak obs

In [ ]:
sfrobs = np.genfromtxt(os.path.join(model_ws, mname + '.sfr.obs.csv'), delimiter=',', names=True)
lakobs = np.genfromtxt(os.path.join(model_ws, mname + '.lak.obs.csv'), delimiter=',', names=True)
lakobs.dtype

### Create mapping array for lake data

In [ ]:
lakeconn = gwf.lak.connectiondata.get_data()
lakmap = {0: [], 1: []}
for v in lakeconn:
    if v['claktype'].upper() == 'VERTICAL':
        cid = v['cellid']
        lakmap[v['lakeno']].append((0, cid[1], cid[2]))

In [ ]:
"""
add lake values to the head array
"""
def sub_lake(h, ls, tmap):
    for key, value in tmap.items():
        s = ls[key]
        for loc in value:
            h[loc] = s
    return h    

#### Retrieve the heads

In [ ]:
hobj = flopy.utils.HeadFile(os.path.join(model_ws, mname + '.hds'))
times= hobj.get_times()

#### Plot the heads with the lake stages

Plot the first and last times in the head file.

In [ ]:
h = hobj.get_data(totim=times[0])
ls = [lakobs['LAK1S'][0], lakobs['LAK2S'][0]]
h = sub_lake(h, ls, lakmap)
pmv = flopy.plot.PlotMapView(gwf)
pmv.plot_array(h, masked_values=[1e+30])
c = pmv.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d')
plt.title('time = {}'.format(times[0]));

In [ ]:
h = hobj.get_data(totim=times[-1])
ls = [lakobs['LAK1S'][-1], lakobs['LAK2S'][-1]]
h = sub_lake(h, ls, lakmap)
pmv = flopy.plot.PlotMapView(gwf)
pmv.plot_array(h, masked_values=[1e+30])
c = pmv.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d')
plt.title('time = {}'.format(times[-1]));

#### Plot the SFR stage results

In [ ]:
names = [name for name in sfrobs.dtype.names[1:] if name[-1] == 'S']
f, axes = plt.subplots(nrows=1, ncols=len(names), figsize=(len(names)*5, 5))
axes = axes.flatten()
for idx, name in enumerate(names):
    axes[idx].plot(sfrobs['time'], sfrobs[name], marker='.', label=name)
    axes[idx].legend();

#### Plot the SFR Q results

In [ ]:
names = [name for name in sfrobs.dtype.names[1:] if name[-1] == 'Q']
f, axes = plt.subplots(nrows=1, ncols=len(names), figsize=(len(names)*5, 5))
axes = axes.flatten()
for idx, name in enumerate(names):
    axes[idx].plot(sfrobs['time'], sfrobs[name], marker='.', label=name)
    axes[idx].legend();

#### Plot the lake results

In [ ]:
names = [name for name in lakobs.dtype.names[1:] if name[-1] == 'S']
f, axes = plt.subplots(nrows=1, ncols=len(names), figsize=(len(names)*5, 5))
axes = axes.flatten()
for idx, name in enumerate(names):
    axes[idx].plot(lakobs['time'], lakobs[name], marker='.', label=name)
    axes[idx].legend();

#### Class exercise 2

Empty code blocks to load and plot the multi-aquifer head results

## Class exercises

---------------------

### Exercise 1: Modify the SFR model to use the time series file

Uncomment the code block above to enable use of the time varying inflow rate.

### Exercise 2: Add a multi-aquifer well to the model


##### packagedata
There are two multi-aquifer wells to the model. Both wells have a radius of 0.5 m and each well will have 4 aquifer connections.

##### connection data
The wells are in layers 4 to 8 in cells (row, column):

    (16, 20) 
    (24, 4)

The top and bottom of each screen in each connected layer can be set to 100. and -100, respectively.

##### stress_period_data
The wells start pumping in stress period 2 with a pumping rate -100,000 for each well. 

##### Add observations
Add a `head` observation in each multi-aquifer well.

##### Plot multi-aquifer well observations
Use SFR stage plots as an example to plot MAW head observations.

### Exercise 3: Move the water extracted from MAW to SFR using the mover


Move water from multi-aquifer well 1 to lake 1.